In [1]:
!python -V

Python 3.10.14


In [1]:
# import libraries

import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# import libraries for training

from sklearn.feature_extraction import DictVectorizer 
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error 

In [4]:
# Read parquet file
df = pd.read_parquet('yellow_tripdata_2023-01.parquet')

## Answer 1

In [5]:
# Check quantity of columns
len(df.columns)

19

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [7]:
# Calculate duration feature 

df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)


## Answer 2

In [8]:
# Calculate standard deviation
df.duration.std()

42.59435124195458

## Answer 3

In [9]:
# length before removing outliers
len(df)

3066766

In [10]:
# capturing only duration of 1 or less than 60 minutes
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [11]:
# length after removing outliers
len(df)

3009173

In [12]:
# Calculating percentage of left after removed data 
percentage_kept = (3009173 / 3066766)*100
print('The percentage left after droping:', round(percentage_kept))

The percentage left after droping: 98


## Answer 4

In [13]:
# creating categorical variables 

categorical = ['PULocationID', 'DOLocationID']

# reading categorical column as strings
df[categorical] = df[categorical].astype(str)

In [14]:
# Checking the change 
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3009173 entries, 0 to 3066765
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           object        
 8   DOLocationID           object        
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee            floa

In [15]:
# creating a dictionary
train_dicts = df[categorical].to_dict(orient='records')

In [16]:
# creating the vectorizer
dv = DictVectorizer()

# creating the feature matrix
X_train = dv.fit_transform(train_dicts)

In [17]:
#Checking the columns
X_train.shape

(3009173, 515)

In [12]:
print('The number of columns is:', (X_train.shape)[1])

The number of columns is: 515


## Answer 5

In [18]:
# Assigning target value
target = 'duration'
y_train = df[target].values

# Creating the model 
lr = LinearRegression()

# fitting the model
lr.fit(X_train, y_train)

# Calculating the MSE
y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

c:\Users\rcata\anaconda3\envs\mlops_zoomcamp\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.649261929201487

## Answer 6

In [3]:
def read_dataframe(filename):
    ''' Function to read from the filename as CSV or parquet file '''
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [4]:
# Reading train data and validation data
df_train = read_dataframe('yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('yellow_tripdata_2023-02.parquet')

In [5]:
# Sanity check 
len(df_train), len(df_val)

(3009173, 2855951)

In [8]:
# processing validation data
categorical = ['PULocationID', 'DOLocationID']
df_train[categorical] = df_train[categorical].astype(str)
df_val[categorical] = df_val[categorical]

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [9]:
# training both training and validation
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [10]:
# calculating MSE
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

c:\Users\rcata\anaconda3\envs\mlops_zoomcamp\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.811819793542861